In [5]:
import os
import pandas as pd
import pinecone
from pinecone_datasets import Dataset
from langchain.embeddings.openai import OpenAIEmbeddings

from misc.tools import *
from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv())

# Set Slack API credentials
OpenAI_apikey = os.environ["OPENAI_APIKEY"]
PINECONE_APIKEY = os.environ["PINECONE_APIKEY"]
PINECONE_ENVIRONMENT = os.environ["PINECONE_ENVIRONMENT"]

In [6]:
df = read_db_to_dataframe("./database/product_database.db")
df.head()

,id,name,description,color,brandName,url,price
0,1,14 in. W x 69 in. H TailorMade Vinyl 4-Board J...,Add value and interest to the exterior of your...,,Builders Edge,https://www.homedepot.com/p/Builders-Edge-14-i...,179.46
1,2,FramePro 3-1/2 in. 34-Degree Framing Nailer,"Designed for tough work environments, this dur...",,Senco,https://www.homedepot.com/p/Senco-FramePro-3-1...,399.00
2,3,38.75 in. x 15.5 in. x 3.125 in. Frameless Non...,Clearly Secure Glass Block Windows come fully ...,,Clearly Secure,https://www.homedepot.com/p/Clearly-Secure-38-...,117.30
3,4,1 gal. #QE-12 Dune Drift Satin Enamel Exterior...,BEHR ULTRA Exterior Paint & Primer delivers ex...,,BEHR ULTRA,https://www.homedepot.com/p/BEHR-ULTRA-1-gal-Q...,53.98
4,5,1 qt. #P240-5 Cheese Puff Extra Durable Eggshe...,BEHR ULTRA SCUFF DEFENSE Stain-Blocking Paint ...,,BEHR ULTRA,https://www.homedepot.com/p/BEHR-ULTRA-1-qt-P2...,20.98


In [7]:
# Read data from the database into a Pandas DataFrame
df = read_db_to_dataframe("./database/product_database.db")
df['content'] = df.apply(lambda row: f'product_name: {row["name"]}, brand: {row["brandName"]}, description: {row["description"]}, color: {row["color"]}, price: {row["price"]} USD, URL: {row["url"]}', axis=1)
df['content']

0        product_name: 14 in. W x 69 in. H TailorMade V...
1        product_name: FramePro 3-1/2 in. 34-Degree Fra...
2        product_name: 38.75 in. x 15.5 in. x 3.125 in....
3        product_name: 1 gal. #QE-12 Dune Drift Satin E...
4        product_name: 1 qt. #P240-5 Cheese Puff Extra ...
                               ...                        
17613    product_name: 84 in. x 84 in. "Indigo Sundown ...
17614    product_name: 6 mm x 1 in. 316 Stainless Steel...
17615    product_name: 24 in. x 84 in. Hollow Core Weat...
17616    product_name: Kids Headphones Volume Limiter M...
17617    product_name: Window Assembly - Front Right, b...
Name: content, Length: 17618, dtype: object

# Embedding

In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings

openai_api_key = OpenAI_apikey
model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=openai_api_key
)

In [10]:
index_name = 'langchain-retrieval-agent'

index_name = 'test-bot'
pinecone.init(
    api_key=PINECONE_APIKEY,
    environment=PINECONE_ENVIRONMENT
)

index = pinecone.Index(index_name=index_name)

In [12]:
from tqdm.auto import tqdm
from uuid import uuid4

batch_size = 100

texts = []
metadatas = []

for i in tqdm(range(0, len(df), batch_size)):
    # get end of batch
    i_end = min(len(df), i+batch_size)
    batch = df.iloc[i:i_end]
    # first get metadata fields for this record
    metadatas = [{
        'name': record['name'],
        'content': record['content']
    } for j, record in batch.iterrows()]
    # get the list of contexts / documents
    documents = batch['content']
    # create document embeddings
    embeds = embed.embed_documents(documents)
    # get IDs
    ids = batch['id'].astype(str)
    # add everything to pinecone
    index.upsert(vectors=zip(ids, embeds, metadatas))

  0%|          | 0/177 [00:00<?, ?it/s]

100%|██████████| 177/177 [09:59<00:00,  3.39s/it]


In [8]:
index.describe_index_stats()

NameError: name 'index' is not defined

# Querying

In [178]:
from langchain.vectorstores import Pinecone

text_field = "text"

# switch back to normal index for langchain
index = pinecone.Index(index_name)

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

In [194]:
query_questions = [
    "white paint used for house exterior",
]

query_vectors = [embed.embed_query(str(question)) for question in query_questions]

query_results = index.query(queries=query_vectors, top_k=5, include_values=False)
query_results


{'results': [{'matches': [{'id': '9540', 'score': 0.876684248, 'values': []},
                          {'id': '17419', 'score': 0.874740958, 'values': []},
                          {'id': '3997', 'score': 0.874060512, 'values': []},
                          {'id': '13015', 'score': 0.871846437, 'values': []},
                          {'id': '2602', 'score': 0.869699836, 'values': []}],
              'namespace': ''}]}

In [195]:
matches = []
scores = []
for match in query_results['results'][0]['matches']:
    matches.append(match['id'])
    scores.append(match['score'])

# Create DataFrame with only matches and scores
matches_df = pd.DataFrame({'id': matches, 'score': scores})
matches_df = matches_df.astype({'id':'int32'})
matches_df.merge(df,left_on="id",right_on="id")

,id,score,name,description,color,brandName,url,price,content
0,9540,0.876684,1 gal. #BWC-18 Spring White Solid Color House ...,The BEHR Solid Color House and Fence Paint is ...,,BEHR,https://www.homedepot.com/p/BEHR-1-gal-BWC-18-...,35.98,product_name: 1 gal. #BWC-18 Spring White Soli...
1,17419,0.874741,1 qt. PPG1125-2 White Sage Semi-Gloss Exterior...,This Glidden Exterior Latex Paint can be tinte...,,Glidden Premium,https://www.homedepot.com/p/Glidden-Premium-1-...,17.98,product_name: 1 qt. PPG1125-2 White Sage Semi-...
2,3997,0.874061,1 qt. PPG1104-1 White Rock Flat Exterior Latex...,This Glidden Exterior Latex Paint can be tinte...,,Glidden Premium,https://www.homedepot.com/p/Glidden-Premium-1-...,15.98,product_name: 1 qt. PPG1104-1 White Rock Flat ...
3,13015,0.871846,1 gal. #BL-W06 Whispering Waterfall Solid Colo...,The BEHR Solid Color House and Fence Paint is ...,,BEHR,https://www.homedepot.com/p/BEHR-1-gal-BL-W06-...,35.98,product_name: 1 gal. #BL-W06 Whispering Waterf...
4,2602,0.869700,1 gal. #100B-7 Hot Pink Solid Color House and ...,The BEHR Solid Color House and Fence Paint is ...,,BEHR,https://www.homedepot.com/p/BEHR-1-gal-100B-7-...,35.98,product_name: 1 gal. #100B-7 Hot Pink Solid Co...


In [206]:
from langchain.vectorstores import Pinecone

text_field = "content"

# switch back to normal index for langchain
index = pinecone.Index(index_name)

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)
     

In [207]:

query = "white paint?"

vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

[Document(page_content='product_name: 1 qt. #YL-W12 Antique White Semi-Gloss Enamel Interior Stain-Blocking Paint and Primer, brand: BEHR DYNASTY, description: Introducing the best of BEHR Paint. Featuring our most advanced stain repellency and unmatched scuff resistance to keep the surfaces of your projects looking newer longer, and fast-drying technology that will get you back to using your spaces quicker. All with one-coat hide guaranteed in over 1,000 colors. Our most durable, most stain repellent, most scuff resistant, one-coat hide paint ever. Darker colors may require additional dry time between coats. Cooler temperatures or higher humidity may prolong drying time., color: , price: 30.98 USD', metadata={'name': '1 qt. #YL-W12 Antique White Semi-Gloss Enamel Interior Stain-Blocking Paint and Primer'}),
 Document(page_content='product_name: 1 gal. Home Decorators Collection #HDC-NT-08 Papier Blanc Matte Interior Stain-Blocking Paint & Primer, brand: BEHR DYNASTY, description: Intr

# LangChain

In [208]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.2,
 'namespaces': {'': {'vector_count': 17618}},
 'total_vector_count': 17618}

In [228]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

# chat completion llm
llm = ChatOpenAI(
    openai_api_key=openai_api_key,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

# retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)
     

In [229]:
qa.run('i need the best paint for painting my house under 100$')

"Based on the provided information, the best paint option for painting your house under $100 would be the 1 gal. #ICC-100 Eastern Amber Flat Exterior Paint & Primer by BEHR MARQUEE, priced at $62.98 USD. This paint features advanced dirt and fade technology, ensuring a long-lasting and freshly painted look for your home's exterior walls."

In [230]:
qa.run('is the paint available in white?')

'Yes, all of the mentioned paints are available in white.'

In [231]:
qa.run('what is the date for christams')

"I'm sorry, but I don't have access to the current date. However, Christmas is celebrated on December 25th every year."

In [232]:
from langchain.agents import Tool

tools = [
    Tool(
        name='Knowledge Base',
        func=qa.run,
        description=(
            'use this tool when answering general knowledge queries to get '
            'more information about the topic'
        )
    )
]

from langchain.agents import initialize_agent

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

In [233]:
agent.run('i need the best paint for painting my house under 100$')



> Entering new AgentExecutor chain...
{
    "action": "Knowledge Base",
    "action_input": "best paint for painting house under $100"
}
Observation: Based on the given context, the best paint for painting a house under $100 would be the 1 qt. Home Decorators Collection #HDC-MD-17 Minimum Beige Satin Enamel Exterior Paint & Primer by BEHR ULTRA, priced at $25.98 USD. This paint offers excellent durability and stain-blocking properties, and it resists rain in as little as 60 minutes after application. It is suitable for use on siding, trim, fences, doors, and masonry.
Thought:{
    "action": "Final Answer",
    "action_input": "The best paint for painting a house under $100 is the 1 qt. Home Decorators Collection #HDC-MD-17 Minimum Beige Satin Enamel Exterior Paint & Primer by BEHR ULTRA, priced at $25.98 USD. It offers excellent durability, stain-blocking properties, and resists rain in as little as 60 minutes after application. It is suitable for use on siding, trim, fences, doors, 

'The best paint for painting a house under $100 is the 1 qt. Home Decorators Collection #HDC-MD-17 Minimum Beige Satin Enamel Exterior Paint & Primer by BEHR ULTRA, priced at $25.98 USD. It offers excellent durability, stain-blocking properties, and resists rain in as little as 60 minutes after application. It is suitable for use on siding, trim, fences, doors, and masonry.'

In [234]:
agent.run('i need to paint my house. What products should i buy and how much will that cost me?')



> Entering new AgentExecutor chain...
{
    "action": "Knowledge Base",
    "action_input": "best paint for house painting"
}
Observation: There is no one-size-fits-all answer to this question as the best paint for house painting can vary depending on factors such as the surface being painted, the desired finish, and personal preferences. However, some popular and highly-rated paint brands for house painting include BEHR ULTRA, BEHR PREMIUM PLUS, and BEHR PRO. It is recommended to consider factors such as durability, stain-blocking properties, ease of application, and the specific needs of your project when choosing the best paint for your house painting.
Thought:{
    "action": "Final Answer",
    "action_input": "There is no one-size-fits-all answer to this question as the best paint for house painting can vary depending on factors such as the surface being painted, the desired finish, and personal preferences. However, some popular and highly-rated paint brands for house painting 

'There is no one-size-fits-all answer to this question as the best paint for house painting can vary depending on factors such as the surface being painted, the desired finish, and personal preferences. However, some popular and highly-rated paint brands for house painting include BEHR ULTRA, BEHR PREMIUM PLUS, and BEHR PRO. It is recommended to consider factors such as durability, stain-blocking properties, ease of application, and the specific needs of your project when choosing the best paint for your house painting.'

In [ ]:
template = """You are a chatbot. Be kind, detailed and nice. Present the given queried search result in a nice way as answer to the user input. dont ask questions back! just take the given context

{chat_history}
Human: {user_msg}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "user_msg"], 
    template=template
)